# Eurostat
Code to collect and process eurostat data to create the following indicators:

* Private sector R&D workforce
* Business Enterprise R&D (BERD)
* Share if hugh growth firms 

Raw data collected using the Eurostat API via the `EuroStat API Client` python package (https://pypi.org/project/eurostatapiclient/).

## Preamble

In [ ]:
from eurostatapiclient import EurostatAPIClient

import numpy as np
import pandas as pd

In [ ]:
VERSION = 'v2.1'
FORMAT = 'json'
LANGUAGE = 'en'

In [ ]:
client = EurostatAPIClient(VERSION, FORMAT, LANGUAGE)

### Mappings

In [ ]:
nuts2_map = {
    'UKC1': 'Tees Valley and Durham',
    'UKC2': 'Northumberland and Tyne and Wear',
    'UKD1': 'Cumbria',
    'UKD6': 'Cheshire',
    'UKD3': 'Greater Manchester',
    'UKD4': 'Lancashire',
    'UKD7': 'Merseyside',
    'UKE1': 'East Riding and North Lincolnshire', 
    'UKE2': 'North Yorkshire',
    'UKE3': 'South Yorkshire',
    'UKE4': 'West Yorkshire', 
    'UKF1': 'Derbyshire and Nottinghamshire',
    'UKF2': 'Leicestershire, Rutland and Northamptonshire', 
    'UKF3': 'Lincolnshire', 
    'UKG1': 'Herefordshire, Worcestershire and Warwickshire',
    'UKG2': 'Shropshire and Staffordshire',
    'UKG3': 'West Midlands', 
    'UKH1': 'East Anglia', 
    'UKH2': 'Bedfordshire and Hertfordshire',
    'UKH3': 'Essex',
    'UKI3': 'Inner London - West', 
    'UKI4': 'Inner London - East',
    'UKI5': 'Outer London - East and North East',
    'UKI6': 'Outer London - South', 
    'UKI7': 'Outer London - West and North West',
    'UKJ1': 'Berkshire, Buckinghamshire, and Oxfordshire', 
    'UKJ2': 'Surrey, East and West Sussex',
    'UKJ3': 'Hampshire and Isle of Wight', 
    'UKJ4': 'Kent', 
    'UKK1': 'Gloucestershire, Wiltshire and Bristol/Bath area', 
    'UKK2': 'Dorset and Somerset', 
    'UKK3': 'Cornwall and Isles of Scilly',
    'UKK4': 'Devon',
    'UKL1': 'West Wales and The Valleys',
    'UKL2': 'East Wales', 
    'UKM2': 'Eastern Scotland', 
    'UKM3': 'South Western Scotland', 
    'UKM5': 'North Eastern Scotland',
    'UKM6': 'Highlands and Islands', 
    'UKN0': 'Northern Ireland', 
    'UKZZ': 'Extra-regio NUTS 2'
    
}

In [ ]:
vars_map = {
    'EUR_HAB': 'euro per inhabitant',
    'MIO_EUR': 'euros (Millions)',
    'FTE': 'full time equivalent (FTE)',
    'HC': 'head count(HC)', 
    'PC_ACT_FTE': 'percentage of active population- FTE',
    'PC_ACT_HC': 'percentage of active population- HC'
}

## Data Collection, Processing & Transformation

This section is made up with three sections- one for each indicator. Each section is broken down in the following steps:

* Use the python package to pull down flattened data by entering using a query & put into a dataframe
* Collect the subset for UK NUTS2 regions
* Replace the codes with the label associated 
* Data is transformed into a pivot table to output the desired format

### Private sector R&D workforce data

In [ ]:
#pull in data
data_priv_nuts2 = client.get_dataset('rd_p_persreg?sinceTimePeriod=2012&geoLevel=nuts2&precision=1&sex=T&sectperf=BES&prof_pos=TOTAL&unit=FTE&unit=HC&unit=PC_ACT_FTE&unit=PC_ACT_HC')

print(data_priv_nuts2.label)

dataframe_priv_nuts2 = data_priv_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
dataframe_priv_nuts2_uk = dataframe_priv_nuts2[dataframe_priv_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
dataframe_priv_nuts2_uk['time'] = dataframe_priv_nuts2_uk['time'].astype(int)
dataframe_priv_nuts2_uk['geo_name'] = dataframe_priv_nuts2_uk['geo'].map(nuts2_map)
dataframe_priv_nuts2_uk['unit'] = dataframe_priv_nuts2_uk['unit'].map(vars_map)

In [ ]:
#pivot table
d_priv = dataframe_priv_nuts2_uk.pivot_table(index=['geo_name','geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo_name')

In [ ]:
#save data
d_priv.to_csv('../../data/processed/eurostat/eurostat_private_rd_workforce_data.csv')

### Business Enterprise R&D (BERD) data

In [ ]:
#pull in data
data_berd_nuts2 = client.get_dataset('rd_e_gerdreg?sinceTimePeriod=2012&geoLevel=nuts2&precision=1&sectperf=BES&unit=EUR_HAB&unit=MIO_EUR')

print(data_berd_nuts2.label)

dataframe_berd_nuts2 = data_berd_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
dataframe_berd_nuts2_uk = dataframe_berd_nuts2[dataframe_berd_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
dataframe_berd_nuts2_uk['time'] = dataframe_berd_nuts2_uk['time'].astype(int)
dataframe_berd_nuts2_uk['geo_name'] = dataframe_berd_nuts2_uk['geo'].map(nuts2_map)
dataframe_berd_nuts2_uk['unit'] = dataframe_berd_nuts2_uk['unit'].map(vars_map)

In [ ]:
#pivot table
d_berd = dataframe_berd_nuts2_uk.pivot_table(index=['geo_name','geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo_name')

In [ ]:
#save data
d_berd.to_csv('../../data/processed/eurostat/eurostat_berd_data.csv')

### Private non-profit (PNPERD) data

In [ ]:
#pull in data
data_pnpberd_nuts2 = client.get_dataset('rd_e_gerdreg?sinceTimePeriod=2012&geoLevel=nuts2&precision=2&sectperf=PNP&unit=EUR_HAB&unit=MIO_EUR')

print(data_pnpberd_nuts2.label)

df_pnpberd_nuts2 = data_pnpberd_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
df_pnpberd_nuts2_uk = df_pnpberd_nuts2[df_pnpberd_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
df_pnpberd_nuts2_uk['time'] = df_pnpberd_nuts2_uk['time'].astype(int)
df_pnpberd_nuts2_uk['geo_name'] = df_pnpberd_nuts2_uk['geo'].map(nuts2_map)
df_pnpberd_nuts2_uk['unit'] = df_pnpberd_nuts2_uk['unit'].map(vars_map)

In [ ]:
d_pnpberd = df_pnpberd_nuts2_uk.pivot_table(index=['geo_name','geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo_name')

In [ ]:
d_pnpberd.to_csv('../../data/processed/eurostat/eurostat_private_non_profit_rd_workforce_data.csv')

### Higher Education Performed R&D expenditure (HERD)

In [ ]:
#pull in data
data_herd_nuts2 = client.get_dataset('rd_e_gerdreg?sinceTimePeriod=2012&geoLevel=nuts2&precision=2&sectperf=HES&unit=EUR_HAB&unit=MIO_EUR')

print(data_herd_nuts2.label)

df_herd_nuts2 = data_herd_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
df_herd_nuts2_uk = df_herd_nuts2[df_herd_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
df_herd_nuts2_uk['time'] = df_herd_nuts2_uk['time'].astype(int)
df_herd_nuts2_uk['geo_name'] = df_herd_nuts2_uk['geo'].map(nuts2_map)
df_herd_nuts2_uk['unit'] = df_herd_nuts2_uk['unit'].map(vars_map)

In [ ]:
d_herd = df_herd_nuts2_uk.pivot_table(index=['geo_name','geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo_name')

In [ ]:
d_herd.to_csv('../../data/processed/eurostat/eurostat_higher_ed_rd_workforce_data.csv')

### Government Performed R&D Expenditure (GovERD)

In [ ]:
#pull in data
data_goverd_nuts2 = client.get_dataset('rd_e_gerdreg?sinceTimePeriod=2012&geoLevel=nuts2&precision=2&sectperf=GOV&unit=EUR_HAB&unit=MIO_EUR')

print(data_goverd_nuts2.label)

df_goverd_nuts2 = data_goverd_nuts2.to_dataframe()

In [ ]:
#UK NUTS2 regions subset
df_goverd_nuts2_uk = df_goverd_nuts2[df_goverd_nuts2['geo'].str.contains('UK')]

In [ ]:
#mappings
df_goverd_nuts2_uk['time'] = df_goverd_nuts2_uk['time'].astype(int)
df_goverd_nuts2_uk['geo_name'] = df_goverd_nuts2_uk['geo'].map(nuts2_map)
df_goverd_nuts2_uk['unit'] = df_goverd_nuts2_uk['unit'].map(vars_map)

In [ ]:
d_goverd = df_goverd_nuts2_uk.pivot_table(index=['geo_name','geo','time'],
               columns = 'unit',
               values = 'values').reset_index().set_index('geo_name')

In [ ]:
d_goverd.to_csv('../../data/processed/eurostat/eurostat_gov_rd_workforce_data.csv')

### Share of high growth firms

In [ ]:
#pull in data
data_share_nuts2 = client.get_dataset('bd_hgnace2_r3?sinceTimePeriod=2012&geoLevel=nuts2&precision=1&indic_sb=V97460&nace_r2=B-E&nace_r2=B-S_X_K642&nace_r2=F&nace_r2=G&nace_r2=H&nace_r2=I&nace_r2=J&nace_r2=K_L_X_K642&nace_r2=M_N&nace_r2=P_Q&nace_r2=R_S')

print(data_share_nuts2.label)

dataframe_share_nuts2 = data_share_nuts2.to_dataframe()

In [ ]:
dataframe_share_nuts2[dataframe_share_nuts2['geo'] == 'UKC1']

In [ ]:
#UK NUTS2 regions subset

dataframe_share_nuts2_uk = dataframe_share_nuts2[dataframe_share_nuts2['geo'].str.contains('UK')]

In [ ]:
dataframe_share_nuts2_uk

In [ ]:
dataframe_share_nuts2_uk.pivot_table(index=['geo','time'],
               columns = 'indic_sb',
               values = 'values')

Note: Does not seem to be UK NUTS2 values for this dataset